In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import requests

# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Semestre 2022-1**<br>
**Profesora:** Paula Aguirre <br>

## Clases 7: Extracción de Datos desde APIs

---

## 1. Extracción de Datos.

Los datos de interés para un problema de ciencia de datos pueden estar ubicados en diversas fuentes: archivos en almacenamiento local o remoto, servidores de bases de datos, y la web. 

En la web, la transmisión de información se realiza mediante el protocolo HTTP (Hypertext Transfer Protocol), diseñado para la comunicación entre los navegadores y servidores web. Sigue el clásico modelo cliente-servidor, en el que un cliente establece una conexión, realizando una petición a un servidor y espera una respuesta del mismo.

![image.png](attachment:image.png)


HTTP define un conjunto de métodos de petición para indicar la acción que se desea realizar para un recurso determinado. Los métodos más comunes son GET y POST:

- `GET`: el método GET  solicita una representación de un recurso específico. Las peticiones que usan el método GET sólo deben recuperar datos.
- `POST`: el método POST se utiliza para enviar una entidad a un recurso en específico, causando a menudo un cambio en el estado o efectos secundarios en el servidor.

En este ejercicio, exploraremos distintas formas de extraer datos desde la WWW, utilizando la librería `requests` para enviar peticiones HTTP a un servidor (Ej. GET, POST), y recoger la respuesta en un objeto tipo `Response`, que implementa métodos y atributos para leer y explorar los datos extraídos. 

La documentación se encuentra en el siguiente link:

https://docs.python-requests.org/en/latest/api/#requests.Response

Siempre que se hace un llamado a `requests.get()`, está ocurriendo dos cosas importantes. Primero, se está construyendo un objeto tipo `Request`, el cual será enviado a un servidor con el fin de obtener información de éste. Segundo, un objeto `Response` es generado una vez que requests obtenga una respuesta del servidor. El objeto respuesta contiene toda la información entregada por el servidor, así como el objecto `Request` que fue creado originalmente.




## 2. API

Una API es un conjunto de protocolos y rutinas que define cómo una aplicación (o programa) se comunica con otros programas, y vice-versa.

Una forma standard de transferir data a través de una API es mediante el format JSON, acrónimo de JavaScript Object Notation. Es un formato interpretable por humanos, y puede ser manejado mediante la librería `json` de python.

Para obtener data de una API, debemos enviar una solicitud o "request". El método `requests.get()` recibe el string de una URL desde donde obtener datos, y argumentos opcionales (keywords) útiles para trabajar con APIs:
- `url`: dirección del recurso, más un string de query o consulta escrito de acuerdo a la documentación de cada API.
- `params`: permite entregar un diccionario con nombres y valores de parámetros para personalizar peticiones a la API.
- `headers`: también corresponde a un diccionario de cabeceras HTTP pasadas a la API, por ejemplo para autenticación del usuario.

El resultado es un objeto `response`, que contiene los datos y la metadata. El método `response.json()` permite acceder solamente a los datos.


### Ejemplo 1: API Open Notify ¿Cuántas personas hay en el espacio hoy?

La API Open Notify entrega información actualizada de la ubicación de la Estación Espacial Internacional (ISS), y de las personas que están en el espacio.

http://open-notify.org/Open-Notify-API/People-In-Space/

Esta es una API sencilla, que no tiene argumentos. Por lo tanto, la consulta que se le enviá es siempre la misma.

In [ ]:
url='http://api.open-notify.org/astros.json'

r=requests.get(url)
r.json()


In [ ]:
df=pd.json_normalize(r.json(),record_path='people')
df

In [ ]:
#¿Dónde está la ISS ahora?
url='http://api.open-notify.org/iss-now.json'
r=requests.get(url)
print(r.json())

In [ ]:
url='http://api.open-notify.org/iss-now.json'
r=requests.get(url)
print(r.json())


### Ejemplo 2: API Yelp Fusion ¿Dónde puedo comer en Santiago?

Por ejemplo, la aplicación Yelp permite a los usuarios calificar y enviar comentarios sobre distintos negocios, y disponibiliza esta información mediante un conjunto de APIs.

https://www.yelp.com/developers/documentation/v3/get_started

Para conectarse a estas APIs, se requiere una clave privada de autentificación (gratuita), que puede ser creada siguiendo las instrucciones en:

https://www.yelp.com/developers/documentation/v3/authentication

En esje ejmplo, usaremos la API *Businesses Search* para obtenre un listado de restaurants en la ciudad de Santiago. La información requerida para hacer la búsqueda está disponible en:

https://www.yelp.com/developers/documentation/v3/business_search

Una característica importante de la API, es que entrega un máximo de 1000 resultados. Además, cada query tiene un límite de 50 resultados. Por lo tanto, es necesario iterar y realizar varias queries para obtener el máximo de 1000 registros. Para esto, se utiliza el parámetro `offset`, que permite ir avanzando en la lista.

In [ ]:
#url de la API
api_url='https://api.yelp.com/v3/businesses/search'

#estos datos corresponden a una cuenta de usuario creada previamente
clientid='GWOCZh9-BmZxtdsAjr7Gug'
apikey='FHVvXoNmTXIl9DuxYis7AV5uLPujm9MLwrhgs5NgvCfaOxd3V6mxt6dQU8eEqYJiGxe816XATx7ufWjbMWqbV-2Uku1jxBJv8BGRC74NroLPl27PDQqs0tDixit-YHYx'
headers={'Authorization':'Bearer %s'%apikey}

In [ ]:
params={'term':'restaurants','location':'RM Santiago, Chile','limit':50}
response=requests.get(api_url,params=params,headers=headers)
data=response.json()
print(data)
#data.keys()
#data['businesses']

En total la base de datos registra 2200 restaurants.

In [ ]:
data['total']

La data entregada por la API Yelp API es un objeto en formato JSON anidado, es decir, un diccionario donde algunos de los valores de atributos corresponden a su vez a listas o diccionarios.

Como vimos anteriormente, podemos llevar estos datos a un formato "aplanado" o "flattened",utilizando la función `json_normalize()` :

https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html

In [ ]:
data['businesses'][0]


In [ ]:
#normalizamos el contenido del diccionario "businesses"
rests = pd.json_normalize(data["businesses"],sep='_',record_path=['categories'],meta=['name','price','rating','review_count','distance',['coordinates','latitude'],['coordinates','longitude'],['location','address1']],errors='ignore')

rests.head()

In [ ]:
rests

In [ ]:
#iteramos
offset=0
allrests=[]
while offset<=950:
    print(offset)
    params={'term':'restaurants','location':'RM Santiago, Chile','limit':50,'offset':offset}

    response=requests.get(api_url,params=params,headers=headers)
    data=response.json()
    #print(data)
    rests = pd.json_normalize(data["businesses"],sep='_',record_path=['categories'],meta=['name','price','rating','review_count','distance',['coordinates','latitude'],['coordinates','longitude'],['location','address1']],errors='ignore')

    allrests.append(rests)
    offset=offset+50
    

In [ ]:
allrests

In [ ]:
rests=pd.concat(allrests,ignore_index=True)
rests

In [ ]:
rests.info()
